- HuggingFacePipeline: inference API를 제공하지 않는 모델을 local에서 다운받아 사용
- huggingFaceEndPoint: hoisting을 제공하기 때문에 서버에 있는 gpu를 사용하여 무료로 사용(local이 아님)

만약 local에서 computing resource가 llm을 감당하지 못 하는 사양이라면, hugging face에서 제공하는 `Dedicated endpoint(전용 엔드포인트)`를 통해 배포 후 사용해볼 수 있다. -> [[🔗 배포 Link](https://huggingface.co/inference-endpoints/dedicated)]

# huggingface model load


huggingface에서 model을 다운받은 경로를 알 수 있다.

```python
# from huggingface_hub import constants
# print(constants.hf_cache_home)
```

In [26]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
  model_id="microsoft/Phi-3-mini-4k-instruct",  
  task="text-generation",
  pipeline_kwargs={
    "max_new_tokens": 200,
    "do_sample": False,
    # "top_k": 50, // do_sample=True일 때 가능한 옵션
    # "temperature": 0.1, // do_sample=True일 때 가능한 옵션
    "repetition_penalty": 1.03
  }
)

chat_model = ChatHuggingFace(llm=llm)

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.26s/it]
Device set to use mps:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [2]:
chat_model.invoke("what is hugging face?")

AIMessage(content="<|user|>\nwhat is hugging face?<|end|>\n<|assistant|>\n Hugging Face is a company that provides AI models and tools for natural language processing (NLP) tasks. It was founded by Facebook AI researchers in 2018, with the goal of making state-of-the-art NLP models accessible to everyone. The company offers pre-trained models, fine-tuning capabilities, and APIs for various NLP tasks such as text generation, translation, summarization, and more.\n\nHugging Face's most popular model is GPT-3, which stands for Generative Pre-trained Transformer 3. This model has been trained on a diverse range of internet text data and can generate human-like text based on given prompts. Other notable models include BERT, RoBERTa, and T5.\n\nThe company also provides an open-source platform called the Hugging Face Hub, where users can share their own models, datasets, and code. This", additional_kwargs={}, response_metadata={}, id='run--b3e3b738-d7cb-4ea7-a33b-389c4ee372f9-0')

In [3]:
# hugging face에서 받은 model 설치 경로
from huggingface_hub import constants

print(constants.hf_cache_home)

/Users/pgj/.cache/huggingface


# 양자화(Quantization)

## Window + Cuda에서 양자화

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
  load_in_4bit=True, # 가중치를 4bit 정수로 압축
  bnb_4bit_quant_type="nf4", # quantization error(양자화 오차)를 줄이는 기법
  # 순전파시 4bit로 저장된 가중치를 실시간으로 풀어(Dequantize on the fly) `float16`으로 변환하여 곱셉/누적을 float16으로 수행
  bnb_4bit_compute_dtype="float16",
  # 1차 양자화에서 생긴 보조 파라미터들을 한 번 더 저비트로 양자화해서 저장(일종의 2단계 압축)하여 메모리/대역폭.
  # 메모리 절감↑ 하지만, 복원 시 미세한 추가 오차가 생길 수 있다.
  bnb_4bit_use_double_quant=True,
)

quantized_llm = HuggingFacePipeline.from_model_id(
  model_id="microsoft/Phi-3-mini-4k-instruct",  
  task="text-generation",
  pipeline_kwargs={
    "max_new_tokens": 200,
    "do_sample": False,
    "repetition_penalty": 1.03
  },
  model_kwargs={
    "quantization_config": quantization_config
  }
)

quantized_chat_model = ChatHuggingFace(llm=quantized_llm)

## Mac + MPS에서 양자화

- bitsandbytes 같은 CUDA 의존 패키지는 맥에서 동작 제한이 있어 다른 방법을 고려해야 한다.  
  => mac에서 양자화하기 [🔗 Link](https://developer.apple.com/kr/videos/play/wwdc2025/298/?time=187)

In [ ]:
# mac에서 mps를 사용한 예제
import torch
from mlx_lm.convert import convert

print("MPS available on this device =>", torch.backends.mps.is_available())

# projection layer & embedding layer는 6bit, 양자화 가능한 layer는 4bit, 양자화 불가능은 False return
def mixed_quantization(layer_path, layer):
  if "lm_head" in layer_path or 'embed_tokens' in layer_path:
    return {"bits": 6, "group_size": 64}
  elif hasattr(layer, "to_quantized"):
    return {"bits": 4, "group_size": 64}
  else:
    return False

# quantization 진행
convert(
  hf_path="microsoft/Phi-3-mini-4k-instruct",
  mlx_path="./models/microsoft-Phi-3-mini-4k-instruct-mixed-4-6-bit",
  dtype="float16",
  quantize=True,
  q_bits=4,
  q_group_size=64,
  quant_predicate=mixed_quantization
)

In [4]:
from mlx_lm import load, generate

model, tokenizer = load("./models/microsoft-Phi-3-mini-4k-instruct-mixed-4-6-bit")
system_prompt = """
  You are an expert in information retrieval and summarization. 
  Your job is to read the provided text and produce a precise, faithful, and concise summary. 
  Prioritize the author’s main claim, key evidence, and conclusions. 
  Use plain English and avoid filler. 
  Do not invent facts that aren’t present in the input.
"""
messages = [{
  "role": "system", "content": system_prompt
}]
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
# response = generate(model, tokenizer, prompt="what is hugging face?", verbose=True)

# Generate the response
response = generate(model, tokenizer, prompt = "what is the hugging face??", verbose=True)




# Answer
Hugging Face is a company and community that provides state-of-the-art pre-trained models and tools for Natural Language Processing (NLP) and computer vision. It offers a wide range of pre-trained models that can be easily integrated into various applications, such as chatbots, text generation, and image recognition. The company also provides a platform for researchers and developers to collaborate and share their work in the field of AI.
Prompt: 8 tokens, 159.601 tokens-per-sec
Generation: 102 tokens, 177.417 tokens-per-sec
Peak memory: 2.324 GB


## langchain과 연계

mlx_lm에서 langchain에서는 `generate_step`을 이용하여 formatter를 전달하지만 mlx_lm에서는 `generate_step`이 존재하지만 mlx_lm.utils에 존재하지 않아 오류가 발생한다. 아마 버전 호환 문제이며 이는 기다려야할 것 같다. 따라서 각 단계를 `runnableLabmda`로 만들어 LCEL문법 기반 chain을 구성함

In [ ]:
from functools import wraps
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from mlx_lm import generate, load

quantized_model_path = "./models/microsoft-Phi-3-mini-4k-instruct-mixed-4-6-bit"
model, tokenizer = load(quantized_model_path)

def runnable_wrapper(func):
  """RunnableLambda wrapper function"""
  @wraps(func)
  def wrapper(*args, **kwargs):
    return RunnableLambda(func)
  
  return wrapper

@runnable_wrapper
def create_chat_prompt(question):
  messages = [
    {
      "role": "system",
      "content": """
        You are an expert in information retrieval and summarization. 
        Your job is to read the provided text and produce a precise, faithful, and concise summary. 
        Prioritize the author’s main claim, key evidence, and conclusions. 
        Use plain English and avoid filler. 
        Do not invent facts that aren’t present in the input.
      """
    },
    {
      "role": "user",
      "content": f"""
        Question: {question}
      """
    }
  ]

  prompt_without_tokenized = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
  prompt_with_tokenized = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

  print("생성된 prompt👇\n", prompt_without_tokenized)

  return prompt_with_tokenized

@runnable_wrapper
def run_llm_with_mlx(prompt):
  return generate(model=model, tokenizer=tokenizer, prompt=prompt)

@runnable_wrapper
def output_parser(answer):
  return answer.replace("<|end|>", "")
  
chain = create_chat_prompt() | run_llm_with_mlx() | output_parser()

Fetching 7 files: 100%|██████████| 7/7 [00:42<00:00,  6.04s/it]


In [25]:
chain.invoke({"question": "what is capital of south korea?"})

'The capital of South Korea is Seoul.<|end|>'

In [20]:
# bitsandbytes 같은 CUDA 의존 패키지는 맥에서 동작 제한이 큽니다
# https://developer.apple.com/kr/videos/play/wwdc2025/298/?time=187

from mlx_lm import load, generate

# hub에서 관련 model을 local에 다운, path는 아래 코드를 통해 확인
# from huggingface_hub import constants
# print(constants.hf_cache_home)
model, tokenizer = load("microsoft/Phi-3-mini-4k-instruct")

# TODO: 올릴 땐 빼기
# print(model) 👉🏻 model에 관련하여 layer들을 보여준다
# print(model.parameters()) 👉🏻 weight, bias등 관련된 parameter에 대한 정보
# print(model.layers[0].self_attn) 👉🏻 특정 layer에도 접근이 가능하다.


# Prepare the prompt for the model
# messages = [{
#   "role": "system", "content": "You are an expert in information retrieval and summarization. Your job is to read the provided text and produce a precise, faithful, and concise summary. Prioritize the author’s main claim, key evidence, and conclusions. Use plain English and avoid filler. Do not invent facts that aren’t present in the input."
# },{
#   "role": "user", "content": "Write one paragraph that summarizes its core message and Provide an unordered list of three key related points"
# }]
messages = [{
  "role": "system", "content": "You are an expert in information retrieval and summarization. Your job is to read the provided text and produce a precise, faithful, and concise summary. Prioritize the author’s main claim, key evidence, and conclusions. Use plain English and avoid filler. Do not invent facts that aren’t present in the input."
}]
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
# response = generate(model, tokenizer, prompt="what is hugging face?", verbose=True)

# Generate the response
response = generate(model, tokenizer, prompt = "what is the hugging face??", verbose=True)




Fetching 13 files: 100%|██████████| 13/13 [00:00<00:00, 168811.00it/s]




# Answer
Hugging Face is a company that specializes in creating and providing AI models and tools for natural language processing (NLP). They offer a wide range of pre-trained models, libraries, and frameworks that can be used for various NLP tasks such as text generation, translation, summarization, and more. Hugging Face's popular library, Transformers, provides a collection of state-of-the-art models that can be easily integrated into applications and projects. The company also focuses on promoting open-source collaboration and community engagement in the field of AI and NLP.
Prompt: 8 tokens, 91.062 tokens-per-sec
Generation: 132 tokens, 60.029 tokens-per-sec
Peak memory: 15.284 GB


# 오류 코드


In [ ]:
from langchain_community.llms.mlx_pipeline import MLXPipeline
from mlx_lm import load

model_path = "./models/microsoft-Phi-3-mini-4k-instruct-mixed-4-6-bit"
model, tokenizer = load(model_path)

llm = MLXPipeline(
  model=model,
  tokenizer=tokenizer
)

llm.invoke("what's my name?")

In [ ]:
# quantization "command" and "python-api"

# Quantization of command in cli
mlx_lm.convert --hf-path "mistralai/Mistral-7B-Instruct-v0.3" \ 👉🏻 모델 위치(출처)
               --mlx-path "./mistral-7b-v0.3-4bit" \ 👉🏻 환/양자화가 끝난 MLX 모델의 저장 위치
               --dtype float16 \ 👉🏻 추론(계산; dequantize on-the-fly) 시  통한 data type지정
               --quantize --q-bits 4 --q-group-size 64 👉🏻 --q-bits 4: 양자화시 4bit로 양자화 진행 / --q-group-size: 가중치 텐서의 특정 축을 따라 ‘몇 개’씩 같은 스케일을 공유할지 정하는 값
               --upload-repo "my-name/mistral-7b-v0.3-4bit"   👉🏻 hugging face에 호스팅 된 model을 다른 사람들과 공유
mlx_lm.convert --hf-path "mistralai/Mistral-7B-Instruct-v0.3" \ 
               --mlx-path "./mistral-7b-v0.3-4bit" \ 
               --dtype float16 \ 
               --quantize --q-bits 4 --q-group-size 64 
               --upload-repo "my-name/mistral-7b-v0.3-4bit"   

# what is tensor in deep learning -> https://towardsdatascience.com/what-is-a-tensor-in-deep-learning-6dedd95d6507/

# 3) 어떻게 선택하나요? (실전 가이드)
# 처음 시작: --q-group-size 64 권장(기본값인 경우가 많음).
# 품질이 아쉽다(특히 수학/코딩/장문 품질): 32로 낮춰 품질↑(오버헤드↑)
# 메모리가 더 급하다: 128로 높여 저장량/대역폭↓(품질 리스크↑)
# 민감 레이어(입출력 프로젝션, LayerNorm 주변 등)는 그룹을 더 촘촘(32)하게 하거나 해당 레이어만 FP16 유지(혼합 정밀)도 실전에서 자주 씁니다.
# 한 줄 요약: 64는 “정확도 손실을 크게 늘리지 않으면서 메타데이터와 커널 효율을 잘 맞춘” 실전형 기본값입니다.

# Quantization with python api
from mlx_lm.convert import convert

# We can choose a different quantization per layer
def mixed_quantization(layer_path, layer, model_config):
  # projection layer, embedding layer는 민감하기 때문에 양자화하는 bit보다 너 높은 bit로 할당한다.
  if "lm_head" in layer_path or "embed_tokens" in layer_path:
    return {"bits": 6, "group_size": 64}
  # 양자화 시 quantization을 지원하지 않는 경우가 있으므로 지원하는 경우만 양자화를 진행할 수 있다.
  elif hasattr(layer, "to_quantized"):
    return {"bits": 4, "group_size": 64}
  else:
    return False

# Convert can be used to change precision, quantize and upload models to Hugging Face
convert(
  hf_path="mistralai/Mistral-7B-Instruct-v0.3",
  mlx_path="./mistral-7b-v0.3-mixed-4-6-bit",
  quantize=True,
  quant_predicate=mixed_quantization
)

str

In [9]:
from huggingface_hub import constants
print(constants.hf_cache_home)

/Users/pgj/.cache/huggingface


# Fine-Tuning with MLX-LM
mlx-lm에서는 `두 가지 typ`을 지원한다.

- `full model fine-tuning`: 사전 학습된 모델의 모든 parameters를 update한다.
- `Low-Rank Adapter training`: 사전 학습된 모델의 weights들은 `freeze`하고 추가 parameters를 학습시켜 `기존 파라미터에 더해져 결과적으로는 가중치가 바뀐 것처럼 동작`한다. 따라서, local에서 fine-tuning을 진행하고자 할 때 메모리 사용이 적고 훨씬 빠르기에 local에서 fine-tuning전략으로 적절하다.

fine-tuning을 할 때 fine-grained로 tuning을 하고 싶을 땐 [[🔗 lora_config.yaml](https://github.com/ml-explore/mlx-examples/blob/9000e280aeb56c2bcce128001ab157030095687a/llms/mlx_lm/examples/lora_config.yaml#L17)]을 통해서 쉽게 설정을 할 수 있다.


👉🏻 fine-tuning command
```shell
$) mlx_lm.lora --train \
            --model "mlx-community/Mistral-7B-Instruct-v0.3-4bit" \
            --data "path/to/our/data/folder" \
            --iters 400 \
            --batch-size 16
```

fine-tuning을 하기 위해 아래의 과정들을 보자(Lora를 통한 `row-rank adapter fine-tuning`)

command를 통해 최근에 Super Bowl에서 누가 이겼냐는 질문에 답은 맞지만 최근 정보로 update되지 않아 최근 정보를 통해서 `fine-tuning`이 필요한 상황이다.

```shell
$) mlx_lm.generate --model mlx-community/Mistral-7B-Instruct-v0.3-4bit --prompt 'Who won the latest Super Bowl'
```

`knowledge cutoff`이후의 super bowl에 대한 data(`question-answer`)를 학습시켜 model을 update할 수 있다. 아래 link에서 `jsonl`의 구조로 `표의 형태`를 하고 있다. 하나의 row를 train data로 주입을 해야 하는데 nlp는 문자열을 통해 학습을 진행하므로 `table:, columns:, Q:, A:`와 같이 `label`을 주어 model이 역할 힌트를 주어 `prompt`와 같이 제공을 할 수 있다. 따라서, 모델 입력은 문자열이기 때문에 이러한 구조를 통해서 train data를 구성한다.
=> [[🔗 train data link](https://github.com/ml-explore/mlx-examples/tree/main/lora/data)]

```shell
$) mlx_lm.lora --train \
               --model mlx-community/Mistral-7B-Instruct-v0.3-4bit \
               --data ./data \
               --iters 300 \
               --batch-size 8 \
               --mask-prompt \
               --learning-rate 1e-5
```

이렇게 fine-tuning을 진행하면 lora를 통해서 adapter가 생성된다. 이 `adapter`를 이용하여 추가적으로 생성된 low rank parameter를 추가하여 답변을 얻을 수 있다.
```shell
$) mlx_lm.generate --model mlx-community/Mistral-7B-Instruct-v0.3-4bit --prompt "Who won the latest Super Bowl" --adapter adapters
```

이제 `최근에 super bowl`에서 이긴 팀을 정확히 구할 수 있다.

> 💡 여기선 `adapter`는 fine-tuning을 통해 나온 `LoRA 체크포인트 경로/ID`이다. 즉 추가 데이터가 있다면 이어서 fine-tuning을 진행하여 parameter를 update할 수 있다.

지금까지 보면 `adapter`와 pre-trained된 가중치가 별도로 존재하는데 이를 `fuse`를 통해서 하나의 model로 구축할 수 있다.

```shell
mlx_lm.fuse --model mlx-community/Mistral-7B-Instruct-v0.3-4bit \
            --adapter-path "path/to/trained/adapters" \
            --save-path "fused-mistral-7B-Instruct-v0.3-4bit" \
            --upload-repo "my-name/fused-mistral-7B-Instruct-v0.3-4bit" 👉🏻 해당 옵션은 Hugging Face에 호스팅 하는 것으로 공유하고자 할 때 넣으면 되는 옵션이다.
```


# fine tuning from gpt answer
## 1) 새로 LoRA 학습 시작 (기존 어댑터 없음)

```shell
mlx_lm.lora --train \
  --model mlx-community/Mistral-7B-Instruct-v0.3-4bit \
  --data ./data --iters 300 --batch-size 8 --learning-rate 1e-5 \
  --mask-prompt
```

> 💡 학습이 끝나면 어댑터(LoRA) 체크포인트 디렉터리가 생성됨

## 2) 이전 LoRA를 기반으로 계속 학습(질문하신 케이스)

같은 베이스 모델 위에서, 과거에 만든 어댑터를 불러와 이어 학습하려면:

```shell
mlx_lm.lora --train \
  --model mlx-community/Mistral-7B-Instruct-v0.3-4bit \
  --adapter ./adapters/mistral-7b-lora  # ← 기존 LoRA 디렉터리 or HF repo
  --data ./data_more --iters 200 --batch-size 8 --learning-rate 5e-6 \
  --mask-prompt
```

이렇게 하면 원본 가중치(베이스)는 freeze 유지, 어댑터 파라미터만 추가 업데이트합니다.
> 💡 주의: 어댑터는 학습 당시의 베이스 체크포인트와 동일한 베이스에 얹어야 합니다(동일 버전/양자화 설정). 다르면 가중치 대응이 깨집니다.


## 3) 추론에서 LoRA 적용
```shell
mlx_lm.generate \
  --model mlx-community/Mistral-7B-Instruct-v0.3-4bit \
  --adapter ./adapters/mistral-7b-lora \
  --prompt "Explain LoRA in simple terms."
```

## 4) 머지(merge)해서 단일 모델로 내보내기(선택)

어댑터를 베이스에 합쳐 단일 체크포인트로 만들 때:

```shell
mlx_lm.merge \
  --model mlx-community/Mistral-7B-Instruct-v0.3-4bit \
  --adapter ./adapters/mistral-7b-lora \
  --out ./merged-mistral7b
```


머지하면 `실제 가중치` 𝑊^ = 𝑊 + Δ𝑊로 저장되어 추론 시 --adapter 없이도 사용 가능.
> 💡 단, 양자화 베이스(int4) 위에서 학습한 LoRA를 머지할 수 있는지는 툴 버전에 따라 제약이 있을 수 있어, 보통은 어댑터로 유지하고 런타임 합성이 일반적입니다.



# Quantization Fundamentals with Hugging Face From "DeepLearning.AI"

free course => https://www.deeplearning.ai/short-courses/quantization-fundamentals-with-hugging-face/

# Lecture 1

- LLM is so huge that hard to run on consumer grade hardware
  => LLM은 너무 거대해서 일반 소비자용 하드웨어에서 돌리기 힘들다.

이를 가능하게 하는 것이 quantization(양자화)이며 다양한 data type option(int8, float16, bfloat16)을 활용하여 model을 압축할 수 있다. 32bit 부동소수점(float)를 저장하고 압축하는 방법과 기술적인 세부사항을 접목시킨다. 

quantization이 어떻게 동작하는지를 small-text generation model에 hugging face의 Quanto library(linear quantization을 어떤 pytorch model에서 사용하기 쉽게 만든 library)를 사용하여 배울 것이다. 

transformer library를 이용하여 model을 load하고 quantization(양자화)을 진행하기 위해 Quanto library를 사용한다.

# Lecture2
model들이 거대해짐에 따라 quantization(양자화)를 통해 model을 축소하여 성능저하(with no little performance gradation)없이 local에서 돌릴 수 있다.

AI model들의 크기가 거대해짐에 비해 hardware에서 처리할 수 있는 정도의 크기와 점점 괴리가 발생한다. 



In [6]:
import torch

# Information of `8-bit unsigned integer`
print(torch.iinfo(torch.uint8))

# Information of `8-bit (signed) integer`
print(torch.iinfo(torch.int8))

# Information of `64-bit (signed) integer`
print(torch.iinfo(torch.int64))

# Information of `32-bit (signed) integer`
print(torch.iinfo(torch.int32))


iinfo(min=0, max=255, dtype=uint8)
iinfo(min=-128, max=127, dtype=int8)
iinfo(min=-9.22337e+18, max=9.22337e+18, dtype=int64)
iinfo(min=-2.14748e+09, max=2.14748e+09, dtype=int32)
